# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846886256549                   -0.70    4.8         
  2   -7.852327455520       -2.26       -1.53    1.0   22.3ms
  3   -7.852614617935       -3.54       -2.56    1.2   24.1ms
  4   -7.852645956845       -4.50       -2.88    2.2   30.3ms
  5   -7.852646489466       -6.27       -3.16    1.2   24.3ms
  6   -7.852646679558       -6.72       -4.05    1.0   22.7ms
  7   -7.852646686385       -8.17       -5.16    1.8   27.6ms
  8   -7.852646686721       -9.47       -5.39    2.0   29.6ms
  9   -7.852646686729      -11.09       -6.10    1.0   24.0ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846829612697                   -0.70           4.5         
  2   -7.852551275026       -2.24       -1.64   0.80    2.2    258ms
  3   -7.852638106373       -4.06       -2.75   0.80    1.0   21.1ms
  4   -7.852646520857       -5.07       -3.31   0.80    2.0   27.5ms
  5   -7.852646670046       -6.83       -4.19   0.80    1.2   23.0ms
  6   -7.852646686386       -7.79       -4.75   0.80    2.0   27.0ms
  7   -7.852646686720       -9.48       -5.52   0.80    1.8   25.0ms
  8   -7.852646686729      -11.01       -6.70   0.80    2.0   27.1ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.441700e+01     3.547406e+00
 * time: 0.058435916900634766
     1     1.143795e+00     1.736075e+00
 * time: 0.27963709831237793
     2    -1.728728e+00     2.032573e+00
 * time: 0.3028559684753418
     3    -3.863084e+00     1.934241e+00
 * time: 0.3361549377441406
     4    -5.366733e+00     1.620328e+00
 * time: 0.3690929412841797
     5    -6.880865e+00     8.697902e-01
 * time: 0.4021189212799072
     6    -6.931228e+00     1.750026e+00
 * time: 0.42505693435668945
     7    -7.369747e+00     1.453064e+00
 * time: 0.4477660655975342
     8    -7.606925e+00     7.165111e-01
 * time: 0.4809730052947998
     9    -7.730732e+00     2.245087e-01
 * time: 0.5138969421386719
    10    -7.773533e+00     9.825509e-02
 * time: 0.5369319915771484
    11    -7.801138e+00     1.239597e-01
 * time: 0.5601930618286133
    12    -7.826568e+00     1.384280e-01
 * time: 0.6403560638427734
    13    -7.841151e+00     5.907131e-02
 * time: 0.663350

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846910556317                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645940433                   -1.64         
  2   -7.852646686730       -6.13       -3.72    1.90s
  3   -7.852646686730      -13.34       -7.27    143ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 5.772794884582083e-7
|ρ_newton - ρ_scfv| = 2.4271451935977816e-7
|ρ_newton - ρ_dm|   = 9.866853200317049e-10
